# 05 - Budget Optimization

Given a total media budget, **budget optimization** finds the channel allocation that
maximizes predicted GMV — using the posterior distribution of channel contributions
from the fitted MMM.

This notebook covers:
1. Current allocation baseline — where is the budget going today?
2. Optimal allocation — where *should* it go to maximize GMV?
3. Three scenarios: conservative (−20%), base (same), aggressive (+30%)
4. ROI comparison across channels
5. Results saved to `outputs/optimization/`

> **Note on reliability:** Budget optimization is only meaningful when the fitted model
> has converged. With a prior-dominated model, optimization results reflect
> prior assumptions more than data. See notebooks 03 and 04 for diagnostics.
> Results here should be treated as **illustrative of the workflow**, not prescriptive.

In [ ]:
import warnings
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings("ignore")
sns.set_theme(style="whitegrid")
%matplotlib inline

from pymc_marketing.mmm import MMM

from mmm_demo.config import OUTPUTS_DIR, ModelConfig
from mmm_demo.data import load_mmm_weekly_data
from mmm_demo.diagnostics import check_convergence

---
## 1. Load Model and Data

> **Prerequisite:** Run notebooks 02, 03, and 04 first.

In [ ]:
df = load_mmm_weekly_data()
config = ModelConfig()
feature_cols = [config.date_column, *config.channel_columns, *config.control_columns]
X = df[feature_cols]
y = df[config.target_column]

# Load most recent saved model
model_dir = OUTPUTS_DIR / "models"
model_files = sorted(model_dir.glob("mmm_fit_*.nc"))
if not model_files:
    raise FileNotFoundError("No saved model found. Run notebook 02 first.")

model_path = model_files[-1]
print(f"Loading: {model_path.name}")
mmm = MMM.load(str(model_path))
idata = mmm.idata
print(
    f"Loaded. Chains: {idata.posterior.dims['chain']}, Draws: {idata.posterior.dims['draw']}"
)

---
## 2. Diagnostic Gate Check

In [ ]:
diag = check_convergence(idata)

print(f"Convergence: {'PASSED' if diag.passed else 'FAILED'}")
print(f"  Max R-hat:   {diag.max_rhat:.4f}")
print(f"  Min ESS:     {diag.min_ess:.0f}")
print(f"  Divergences: {diag.divergences}")
print()

if not diag.passed:
    print("WARNING: Model has not converged.")
    print("Optimization results below are illustrative of the workflow.")
    print("In production, only optimize budgets with a converged model.")

---
## 3. Current Allocation Baseline

Before optimizing, understand the current spend distribution — both in absolute terms
and as a percentage of total media investment.

In [ ]:
# Current weekly spend statistics
current_spend = df[config.channel_columns]
total_weekly_budget = current_spend.mean().sum()
total_annual_budget = current_spend.sum().sum()

baseline = pd.DataFrame(
    {
        "mean_weekly": current_spend.mean().round(0),
        "total_annual": current_spend.sum().round(0),
        "share_%": (current_spend.mean() / current_spend.mean().sum() * 100).round(1),
        "min_weekly": current_spend.min().round(0),
        "max_weekly": current_spend.max().round(0),
    }
).sort_values("share_%", ascending=False)

print("Current budget statistics:")
print(f"  Mean weekly total:  {total_weekly_budget:>15,.0f} INR")
print(f"  Annual total:       {total_annual_budget:>15,.0f} INR")
print()
print("Per-channel breakdown:")
print(baseline.to_string())

In [ ]:
# Visualize current allocation
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

colors = plt.cm.Set2(np.linspace(0, 1, len(config.channel_columns)))

# Bar chart: mean weekly spend
baseline["mean_weekly"].plot.bar(
    ax=axes[0], color=colors[: len(baseline)], edgecolor="black"
)
axes[0].set_title("Current Mean Weekly Spend by Channel")
axes[0].set_ylabel("INR")
axes[0].tick_params(axis="x", rotation=45)

# Pie chart: share
baseline["share_%"].plot.pie(
    ax=axes[1], autopct="%1.1f%%", colors=colors[: len(baseline)], startangle=90
)
axes[1].set_title("Current Spend Share")
axes[1].set_ylabel("")

plt.suptitle("Current Media Budget Allocation", fontsize=13)
plt.tight_layout()
plt.show()

---
## 4. Budget Optimization

PyMC-Marketing's optimizer finds the channel allocation that maximizes the
expected total contribution — using the posterior samples to estimate
the response of each channel at different spend levels.

In [ ]:
# Define budget bounds: allow each channel to vary ±80% from current mean
budget_bounds = {
    ch: (current_spend[ch].mean() * 0.20, current_spend[ch].mean() * 1.80)
    for ch in config.channel_columns
}

print("Budget bounds per channel (min / current / max):")
for ch, (lo, hi) in budget_bounds.items():
    cur = current_spend[ch].mean()
    print(f"  {ch:<15}: {lo:>12,.0f} / {cur:>12,.0f} / {hi:>12,.0f}")

print(f"\nTotal weekly budget: {total_weekly_budget:,.0f}")

In [ ]:
# Run budget optimizer
# PyMC-Marketing provides optimize_budget() on the fitted MMM object.
# The exact API may vary by version — we try the most common patterns.

opt_result = None
opt_method = None

# Pattern 1: MMM.optimize_budget() method (recent versions)
if hasattr(mmm, "optimize_budget"):
    try:
        print("Trying mmm.optimize_budget()...")
        opt_result = mmm.optimize_budget(
            budget=total_weekly_budget,
            budget_bounds=budget_bounds,
            num_periods=1,
        )
        opt_method = "mmm.optimize_budget"
        print(f"Success via {opt_method}")
    except Exception as e:
        print(f"  mmm.optimize_budget() failed: {e}")

# Pattern 2: optimize_channel_budget_for_maximum_contribution
if opt_result is None and hasattr(
    mmm, "optimize_channel_budget_for_maximum_contribution"
):
    try:
        print("Trying mmm.optimize_channel_budget_for_maximum_contribution()...")
        raw = mmm.optimize_channel_budget_for_maximum_contribution(
            total_budget=total_weekly_budget,
            channels=config.channel_columns,
            budget_bounds=budget_bounds,
        )
        opt_result = raw
        opt_method = "optimize_channel_budget_for_maximum_contribution"
        print(f"Success via {opt_method}")
    except Exception as e:
        print(f"  Failed: {e}")

if opt_result is None:
    print()
    print(
        "Automatic optimization not available with the current pymc-marketing version."
    )
    print("Using manual response-curve based allocation below.")

In [ ]:
# Manual allocation using posterior mean saturation parameters
# Strategy: allocate more to channels with higher marginal returns at current spend levels
import arviz as az
from scipy.special import expit

summary_lam = az.summary(idata, var_names=["saturation_lam"])
summary_beta = az.summary(idata, var_names=["saturation_beta"])


def saturation_response(spend, lam, beta):
    """LogisticSaturation response (PyMC-Marketing convention)."""
    return beta * (2 * expit(lam * spend) - 1)


def marginal_response(spend, lam, beta):
    """Derivative of LogisticSaturation at current spend level."""
    s = expit(lam * spend)
    return 2 * beta * lam * s * (1 - s)


# Compute marginal returns at current spend (MaxAbsScaled)
channel_max_spend = current_spend.max()  # MaxAbsScaler reference
scaled_mean_spend = current_spend.mean() / channel_max_spend  # scale to [0,1]

marginal_returns = {}
for ch in config.channel_columns:
    lam = summary_lam.loc[f"saturation_lam[{ch}]", "mean"]
    beta = summary_beta.loc[f"saturation_beta[{ch}]", "mean"]
    x_scaled = float(scaled_mean_spend[ch])
    marginal_returns[ch] = marginal_response(x_scaled, lam, beta)

marginal_df = pd.DataFrame.from_dict(
    marginal_returns, orient="index", columns=["marginal_return"]
).sort_values("marginal_return", ascending=False)

print("Marginal return at current spend level (posterior mean):")
print(marginal_df.to_string())
print()
print("Higher = more incremental GMV per additional unit of spend at current levels")

---
## 5. Three Budget Scenarios

We evaluate three total budget levels, each with proportional reallocation guided by
marginal returns:

| Scenario | Total Budget | Change |
|----------|-------------|--------|
| Conservative | −20% of current | Budget cut, reallocate to highest-return channels |
| Base | Current total | Same spend, rebalance toward higher-return channels |
| Aggressive | +30% of current | Additional spend directed to highest-return channels |

In [ ]:
# Build allocation scenarios using marginal-return-weighted reallocation
# The marginal return at current spend determines how additional (or reduced) budget is distributed


def allocate_budget(
    total_budget: float, current: pd.Series, marginal: pd.Series, bounds: dict
) -> pd.Series:
    """Allocate total_budget across channels guided by marginal returns.

    Channels with higher marginal returns receive proportionally more budget.
    Allocation is clipped to channel bounds.
    """
    weights = marginal / marginal.sum()
    raw_alloc = weights * total_budget

    # Apply bounds
    clipped = pd.Series(
        {
            ch: np.clip(raw_alloc[ch], bounds[ch][0], bounds[ch][1])
            for ch in current.index
        }
    )

    # Renormalize to hit total budget exactly
    clipped = clipped / clipped.sum() * total_budget
    return clipped


# Current mean weekly spend per channel
current_mean = current_spend.mean()
marginal_series = pd.Series(marginal_returns)

budgets = {
    "Conservative (−20%)": total_weekly_budget * 0.80,
    "Base (current)": total_weekly_budget * 1.00,
    "Aggressive (+30%)": total_weekly_budget * 1.30,
}

allocations = {}
for name, budget in budgets.items():
    alloc = allocate_budget(budget, current_mean, marginal_series, budget_bounds)
    allocations[name] = alloc

allocations_df = pd.DataFrame(allocations)
allocations_df.loc["TOTAL"] = allocations_df.sum()

print("Suggested channel allocations (weekly INR):")
print(allocations_df.to_string(float_format="{:,.0f}".format))

In [ ]:
# Compute expected GMV response for each scenario using saturation curves
def expected_contribution(alloc: pd.Series, max_spend: pd.Series) -> float:
    """Estimate total channel contribution using posterior mean saturation params."""
    total = 0.0
    for ch in alloc.index:
        lam = summary_lam.loc[f"saturation_lam[{ch}]", "mean"]
        beta = summary_beta.loc[f"saturation_beta[{ch}]", "mean"]
        x_scaled = alloc[ch] / max_spend[ch]
        total += saturation_response(x_scaled, lam, beta)
    return total


# Also compute for current allocation
scenarios_results = {}
for name, alloc in allocations.items():
    expected = expected_contribution(alloc, channel_max_spend)
    budget = alloc.sum()
    scenarios_results[name] = {
        "total_budget": budget,
        "expected_contribution_index": expected,
        **alloc.to_dict(),
    }

# Add current as baseline
current_expected = expected_contribution(current_mean, channel_max_spend)
scenarios_results["Current (actual)"] = {
    "total_budget": current_mean.sum(),
    "expected_contribution_index": current_expected,
    **current_mean.to_dict(),
}

results_df = pd.DataFrame(scenarios_results).T
print("Scenario comparison:")
print(
    results_df[
        ["total_budget", "expected_contribution_index"] + config.channel_columns
    ].to_string(float_format="{:,.1f}".format)
)

In [ ]:
# Visualize scenario comparison
channels = config.channel_columns
scenario_names = list(allocations.keys())

fig, axes = plt.subplots(1, 3, figsize=(16, 6))

bar_colors = plt.cm.Set2(np.linspace(0, 1, len(channels)))

# --- Panel 1: channel allocation per scenario ---
scenario_alloc = pd.DataFrame(
    {name: alloc for name, alloc in allocations.items()},
    index=channels,
)
scenario_alloc.T.plot.bar(
    ax=axes[0], stacked=True, color=bar_colors, edgecolor="black", linewidth=0.4
)
axes[0].axhline(
    total_weekly_budget,
    color="black",
    linestyle="--",
    linewidth=1.5,
    label="Current total",
)
axes[0].set_title("Channel Allocation by Scenario")
axes[0].set_ylabel("Weekly spend (INR)")
axes[0].tick_params(axis="x", rotation=30)
axes[0].legend(bbox_to_anchor=(1.02, 1), loc="upper left", fontsize=8)

# --- Panel 2: share of budget per channel per scenario ---
scenario_share = scenario_alloc.div(scenario_alloc.sum()) * 100

# Add current share
current_share = (current_mean / current_mean.sum() * 100).rename("Current")
all_shares = pd.concat(
    [current_share.to_frame().T, scenario_share.T.rename(index=lambda x: x)]
)

all_shares.plot.bar(
    ax=axes[1], stacked=True, color=bar_colors, edgecolor="black", linewidth=0.4
)
axes[1].set_title("Channel Share by Scenario (%)")
axes[1].set_ylabel("% of total budget")
axes[1].tick_params(axis="x", rotation=30)
axes[1].legend(bbox_to_anchor=(1.02, 1), loc="upper left", fontsize=8)

# --- Panel 3: expected contribution index ---
contrib_values = [
    scenarios_results["Current (actual)"]["expected_contribution_index"],
] + [scenarios_results[name]["expected_contribution_index"] for name in scenario_names]
contrib_labels = ["Current"] + [n.split(" ")[0] for n in scenario_names]

bar_c = axes[2].bar(
    contrib_labels,
    contrib_values,
    color=["gray"] + ["steelblue", "seagreen", "coral"],
    edgecolor="black",
)
axes[2].set_title("Expected Contribution Index\n(posterior mean saturation, scaled)")
axes[2].set_ylabel("Contribution index")
axes[2].tick_params(axis="x", rotation=15)

plt.suptitle("Budget Scenario Comparison", fontsize=13)
plt.tight_layout()
plt.show()

---
## 6. ROI Analysis

ROI per channel = expected incremental contribution / spend.
Because contributions are on a MaxAbsScaled index (not raw INR), we use the
**contribution index per 1M INR** as a comparable ROI metric.

In [ ]:
roi_data = []
for ch in config.channel_columns:
    lam = summary_lam.loc[f"saturation_lam[{ch}]", "mean"]
    beta = summary_beta.loc[f"saturation_beta[{ch}]", "mean"]

    # Contribution at current vs zero spend
    x_current = float(current_mean[ch] / channel_max_spend[ch])
    contrib_current = saturation_response(x_current, lam, beta)
    contrib_zero = saturation_response(0.0, lam, beta)
    incremental = contrib_current - contrib_zero

    spend_in_millions = current_mean[ch] / 1e6
    roi = incremental / spend_in_millions if spend_in_millions > 0 else 0.0

    roi_data.append(
        {
            "channel": ch,
            "mean_weekly_spend_INR": current_mean[ch],
            "contribution_index": incremental,
            "roi_per_1M_INR": roi,
        }
    )

roi_df = (
    pd.DataFrame(roi_data)
    .set_index("channel")
    .sort_values("roi_per_1M_INR", ascending=False)
)
print("ROI by channel (contribution index per 1M INR spent weekly):")
print(roi_df.to_string(float_format="{:.4f}".format))
print()
print("Note: these are posterior-mean point estimates. Full Bayesian uncertainty")
print("would show wide HDI bands for each channel — the ranking is not reliable")
print("without convergence.")

fig, ax = plt.subplots(figsize=(9, 4))
roi_df["roi_per_1M_INR"].plot.bar(ax=ax, color="steelblue", edgecolor="black")
ax.set_title("Estimated ROI per 1M INR of Weekly Spend (posterior mean)")
ax.set_ylabel("Contribution index per 1M INR")
ax.tick_params(axis="x", rotation=30)
plt.tight_layout()
plt.show()

---
## 7. Save Results

In [ ]:
opt_dir = OUTPUTS_DIR / "optimization"
opt_dir.mkdir(parents=True, exist_ok=True)

date_str = datetime.now().strftime("%Y-%m-%d")

# Save scenario comparison
scenario_path = opt_dir / f"budget_scenarios_{date_str}.csv"
pd.DataFrame(scenarios_results).T.to_csv(scenario_path)
print(f"Scenario comparison saved to: {scenario_path}")

# Save ROI table
roi_path = opt_dir / f"channel_roi_{date_str}.csv"
roi_df.to_csv(roi_path)
print(f"ROI table saved to:           {roi_path}")

# Save allocation details
alloc_path = opt_dir / f"allocations_{date_str}.csv"
allocations_df.to_csv(alloc_path)
print(f"Allocations saved to:         {alloc_path}")

print()
print("Contents of outputs/optimization/:")
for f in sorted(opt_dir.glob("*.csv")):
    print(f"  {f.name}")

---
## 8. Conclusions

### What this notebook showed

| Step | Finding |
|------|---------|
| **Current allocation** | Sponsorship dominates at 46.6%, TV is smallest at 5.6% |
| **Marginal returns** | Channels differ in marginal return at current spend levels |
| **Scenario analysis** | Conservative/base/aggressive scenarios with rebalancing |
| **ROI ranking** | Channels ranked by estimated incremental GMV per spend unit |

### Important caveats

**These results should NOT be used for actual budget decisions without:**

1. **A converged model** — R-hat < 1.01, ESS > 400. With R-hat > 2, the posterior is
   essentially the prior. Channel rankings reflect prior assumptions, not data.

2. **More data** — 12 effective monthly observations cannot reliably estimate 17 parameters.
   Collecting 2+ years of weekly data or using daily data would dramatically improve reliability.

3. **Uncertainty quantification** — the contribution index used here is a posterior mean.
   In production, use the full posterior to get credible intervals around ROI estimates.

4. **Business constraints** — minimum spend floors, campaign commitments, brand
   presence requirements, and lead times are not captured in this model.

### What a production workflow would add

- Use PyMC-Marketing's `BudgetOptimizer` with the full posterior for proper uncertainty-aware optimization
- Run holdout validation to verify out-of-sample predictive accuracy before trusting ROI estimates
- Compare optimizer recommendations against marketing team's judgment and constraints
- Refresh the model quarterly as new spend data arrives

> This demo illustrates the **workflow** — data → model → diagnostics → attribution → optimization.
> The specific numbers are dataset- and prior-dependent. The methodology is sound.